# Generate a metric

In [ ]:
import pandas as pd
import numpy as np

# Generate a date range for 3 years of daily data
dates = pd.date_range(start='2021-01-01', end='2023-12-31', freq='D')
# Simulate some timeseries data
np.random.seed(42)
values = np.random.randn(len(dates)).cumsum()  # Example: cumulative sum of random values

# Create the timeseries DataFrame
ts_df = pd.DataFrame({'Date': dates, 'Value': values})

# Calculate a general metric: mean and standard deviation to date
mean_value = ts_df['Value'].mean()
std_value = ts_df['Value'].std()

Mean value to date: 2.39
Standard deviation to date: 12.80


In [2]:
# Define 5 custom metrics as functions
def metric_mean(series):
    return series.mean()

def metric_std(series):
    return series.std()

def metric_median(series):
    return series.median()

def metric_max(series):
    return series.max()

def metric_min(series):
    return series.min()

# List of metrics and their names for reporting
metrics = [
    ("Mean", metric_mean),
    ("Std Dev", metric_std),
    ("Median", metric_median),
    ("Max", metric_max),
    ("Min", metric_min)
]

# Example: set timeframes to compare (can be customized)
custom_timeframes = [
    ("2021-01-01", "2021-03-31"),
    ("2022-01-01", "2022-03-31"),
    ("2023-01-01", "2023-03-31"),
    ("2021-07-01", "2021-09-30"),
    ("2023-10-01", "2023-12-31"),
]

# Calculate metrics for each timeframe
results = []
for idx, (start, end) in enumerate(custom_timeframes):
    df_tf = ts_df[(ts_df['Date'] >= start) & (ts_df['Date'] <= end)]
    metric_values = [func(df_tf['Value']) for _, func in metrics]
    results.append([f"Timeframe {idx+1}: {start} to {end}"] + metric_values)

# Create a DataFrame for easy viewing
metrics_df = pd.DataFrame(
    results,
    columns=["Timeframe"] + [name for name, _ in metrics]
)
metrics_df

,Timeframe,Mean,Std Dev,Median,Max,Min
0,Timeframe 1: 2021-01-01 to 2021-03-31,-6.060472,4.763514,-7.935465,4.480611,-12.246656
1,Timeframe 2: 2022-01-01 to 2022-03-31,8.149207,3.024029,7.509933,15.635646,2.790568
2,Timeframe 3: 2023-01-01 to 2023-03-31,-8.846034,3.066044,-7.900965,-4.033624,-16.666107
3,Timeframe 4: 2021-07-01 to 2021-09-30,-2.380047,2.976460,-1.465302,2.639413,-8.154193
4,Timeframe 5: 2023-10-01 to 2023-12-31,31.525194,3.588265,32.017569,37.084619,21.068739


In [3]:
# Function to generate a timeseries with custom mean and std for each timeframe
def generate_custom_timeseries(dates, timeframes, params):
    """
    dates: DatetimeIndex for the full period
    timeframes: list of (start, end) tuples
    params: list of (mean, std) tuples, one per timeframe
    Returns: DataFrame with Date and Value columns
    """
    values = np.zeros(len(dates))
    for (start, end), (mu, sigma) in zip(timeframes, params):
        mask = (dates >= start) & (dates <= end)
        n = mask.sum()
        # Generate values for this timeframe
        values[mask] = np.random.normal(loc=mu, scale=sigma, size=n)
    return pd.DataFrame({'Date': dates, 'Value': values})

# Example: define statistical parameters for 5 new timeseries (mean, std) for each timeframe
stat_params_list = [
    [(0, 1), (2, 1), (4, 1), (6, 1), (8, 1)],      # Series 1: increasing mean
    [(5, 2), (5, 2), (5, 2), (5, 2), (5, 2)],      # Series 2: constant mean/std
    [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)],      # Series 3: increasing std
    [(-2, 1), (0, 1), (2, 1), (0, 1), (-2, 1)],    # Series 4: mean up and down
    [(2, 1), (4, 2), (6, 3), (8, 4), (10, 5)],     # Series 5: both mean and std increase
]

# Generate the 5 new timeseries
custom_series_list = []
for i, params in enumerate(stat_params_list):
    df = generate_custom_timeseries(dates, custom_timeframes, params)
    df['Series'] = f"Series {i+1}"
    custom_series_list.append(df)

# Combine all into one DataFrame for easy plotting/analysis
all_custom_series_df = pd.concat(custom_series_list, ignore_index=True)
all_custom_series_df.head()

,Date,Value,Series
0,2021-01-01,-0.080717,Series 1
1,2021-01-02,0.078635,Series 1
2,2021-01-03,-1.998201,Series 1
3,2021-01-04,0.916328,Series 1
4,2021-01-05,0.346488,Series 1


# Analyze and report

In [4]:
# Create a metrics table for each series in all_custom_series_df

metrics_tables = {}
for series_name in all_custom_series_df['Series'].unique():
    df_series = all_custom_series_df[all_custom_series_df['Series'] == series_name]
    results = []
    for idx, (start, end) in enumerate(custom_timeframes):
        df_tf = df_series[(df_series['Date'] >= start) & (df_series['Date'] <= end)]
        metric_values = [func(df_tf['Value']) for _, func in metrics]
        results.append([f"Timeframe {idx+1}: {start} to {end}"] + metric_values)
    metrics_tables[series_name] = pd.DataFrame(
        results,
        columns=["Timeframe"] + [name for name, _ in metrics]
    )

# Example: display the metrics table for "Series 1"
metrics_tables["Series 1"]

,Timeframe,Mean,Std Dev,Median,Max,Min
0,Timeframe 1: 2021-01-01 to 2021-03-31,0.124424,1.142110,0.190497,2.439752,-2.896255
1,Timeframe 2: 2022-01-01 to 2022-03-31,2.008253,0.962751,2.007223,4.579709,0.006264
2,Timeframe 3: 2023-01-01 to 2023-03-31,3.947170,0.970244,4.019267,6.493000,1.408958
3,Timeframe 4: 2021-07-01 to 2021-09-30,6.220654,1.002035,6.254673,8.601683,4.033643
4,Timeframe 5: 2023-10-01 to 2023-12-31,8.101259,1.055671,7.984555,10.558199,5.127738


In [5]:
# Calculate the correlation between the values of each timeseries across the 5 custom timeframes

series_corrs = {}

for series_name in all_custom_series_df['Series'].unique():
    df_series = all_custom_series_df[all_custom_series_df['Series'] == series_name]
    # For each timeframe, extract the values as a list
    values_by_timeframe = []
    for start, end in custom_timeframes:
        tf_values = df_series[(df_series['Date'] >= start) & (df_series['Date'] <= end)]['Value'].values
        values_by_timeframe.append(tf_values)
    # To compute correlation, all arrays must be the same length; use the minimum length
    min_len = min(len(arr) for arr in values_by_timeframe)
    trimmed = [arr[:min_len] for arr in values_by_timeframe]
    # Stack as columns and compute correlation
    data = np.column_stack(trimmed)
    corr = pd.DataFrame(
        np.corrcoef(data, rowvar=False),
        index=[f"Timeframe {i+1}" for i in range(len(custom_timeframes))],
        columns=[f"Timeframe {i+1}" for i in range(len(custom_timeframes))]
    )
    series_corrs[series_name] = corr

# Example: show the correlation matrix for Series 1
series_corrs["Series 1"]

,Timeframe 1,Timeframe 2,Timeframe 3,Timeframe 4,Timeframe 5
Timeframe 1,1.000000,0.007947,-0.095700,-0.095627,0.074158
Timeframe 2,0.007947,1.000000,-0.081286,0.090134,0.079722
Timeframe 3,-0.095700,-0.081286,1.000000,-0.079972,-0.027009
Timeframe 4,-0.095627,0.090134,-0.079972,1.000000,-0.169452
Timeframe 5,0.074158,0.079722,-0.027009,-0.169452,1.000000


In [9]:
# Calculate the correlation between the values of each series across the 5 custom series for each timeframe

series_names = [f"Series {i+1}" for i in range(5)]
correlations_across_series = {}

for idx, (start, end) in enumerate(custom_timeframes):
    # For each series, extract the values for this timeframe
    values_by_series = []
    for series_name in series_names:
        df_series = all_custom_series_df[all_custom_series_df['Series'] == series_name]
        tf_values = df_series[(df_series['Date'] >= start) & (df_series['Date'] <= end)]['Value'].values
        values_by_series.append(tf_values)
    # To compute correlation, all arrays must be the same length; use the minimum length
    min_len = min(len(arr) for arr in values_by_series)
    trimmed = [arr[:min_len] for arr in values_by_series]
    # Stack as columns and compute correlation
    data = np.column_stack(trimmed)
    corr = pd.DataFrame(
        np.corrcoef(data, rowvar=False),
        index=series_names,
        columns=series_names
    )
    correlations_across_series[f"Timeframe {idx+1}: {start} to {end}"] = corr

# Example: show the correlation matrix for the first timeframe
correlations_across_series["Timeframe 1: 2021-01-01 to 2021-03-31"]

,Series 1,Series 2,Series 3,Series 4,Series 5
Series 1,1.000000,0.087982,-0.213407,0.104350,-0.000549
Series 2,0.087982,1.000000,-0.124899,-0.119029,-0.233384
Series 3,-0.213407,-0.124899,1.000000,-0.141579,0.066368
Series 4,0.104350,-0.119029,-0.141579,1.000000,0.116496
Series 5,-0.000549,-0.233384,0.066368,0.116496,1.000000


## What do we have for plotting?
* all_custom_series_df
* metrics_tables
* series_corrs
* correlations_across_series

In [ ]:
# Convert custom_timeframes from str to datetime objects
custom_timeframes_dt = [
    (pd.to_datetime(start), pd.to_datetime(end))
    for start, end in custom_timeframes
]

[(Timestamp('2021-01-01 00:00:00'), Timestamp('2021-03-31 00:00:00')),
 (Timestamp('2022-01-01 00:00:00'), Timestamp('2022-03-31 00:00:00')),
 (Timestamp('2023-01-01 00:00:00'), Timestamp('2023-03-31 00:00:00')),
 (Timestamp('2021-07-01 00:00:00'), Timestamp('2021-09-30 00:00:00')),
 (Timestamp('2023-10-01 00:00:00'), Timestamp('2023-12-31 00:00:00'))]

In [ ]:
from plotly.subplots import make_subplots

import plotly.graph_objs as go

def plot_series_summary(series_name):
    # 1. Filter data for the selected series
    df_series = all_custom_series_df[all_custom_series_df['Series'] == series_name]
    metrics_df = metrics_tables[series_name].copy()
    corr_df = series_corrs[series_name].copy()
    tf_label = list(correlations_across_series.keys())[0]
    corr_across = correlations_across_series[tf_label].copy()

    # Format metrics table: round numbers and shorten timeframe labels
    metrics_df = metrics_df.round(2)
    metrics_df['Timeframe'] = [f"TF{i+1}" for i in range(len(metrics_df))]
    # Format correlation matrices: round and shorten index/columns
    short_labels = [f"TF{i+1}" for i in range(len(corr_df))]
    corr_df.index = short_labels
    corr_df.columns = short_labels
    corr_df = corr_df.round(2)
    corr_across.index = [f"S{i+1}" for i in range(len(corr_across))]
    corr_across.columns = [f"S{i+1}" for i in range(len(corr_across))]
    corr_across = corr_across.round(2)

    # Define light colors for each timeframe
    tf_colors = [
        "rgba(255, 235, 205, 0.7)",  # light yellow
        "rgba(204, 236, 255, 0.7)",  # light blue
        "rgba(204, 255, 204, 0.7)",  # light green
        "rgba(255, 204, 229, 0.7)",  # light pink
        "rgba(255, 255, 204, 0.7)",  # light cream
    ]

    # 2. Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            f"Timeseries: {series_name}",
            "Metrics Table",
            "Correlation (Timeframes)",
            "Correlation (Across Series)"
        ],
        specs=[[{"type": "scatter"}, {"type": "table"}],
               [{"type": "heatmap"}, {"type": "heatmap"}]]
    )

    # Add background rectangles for each timeframe in the timeseries plot
    # Timeseries plot
    fig.add_trace(
        go.Scatter(
            x=df_series['Date'],
            y=df_series['Value'],
            mode='lines',
            name=series_name
        ),
        row=1, col=1
    )

    for i, (start, end) in enumerate(custom_timeframes_dt):
        fig.add_vrect(
            x0=start, x1=end,
            fillcolor=tf_colors[i % len(tf_colors)],
            opacity=1,
            layer="below",
            line_width=0,
            row=1, col=1
        )

    cell_colors = []
    for col in metrics_df.columns:
        if col == 'Timeframe':
            cell_colors.append(tf_colors)
        else:
            cell_colors.append(['lavender'] * len(metrics_df))

    # Add table trace
    fig.add_trace(
        go.Table(
            header=dict(
                values=list(metrics_df.columns),
                fill_color='paleturquoise',
                align='left'
            ),
            cells=dict(
                values=[metrics_df[col] for col in metrics_df.columns],
                fill_color=cell_colors,
                align='left'
            )
        ),
        row=1, col=2
    )
    # Correlation (Timeframes)
    fig.add_trace(
        go.Heatmap(
            z=corr_df.values,
            x=corr_df.columns,
            y=corr_df.index,
            colorscale='RdBu',
            zmin=-1, zmax=1,
            colorbar=dict(title="Corr"),
            showscale=True
        ),
        row=2, col=1
    )

    # Correlation (Across Series) for first timeframe
    fig.add_trace(
        go.Heatmap(
            z=corr_across.values,
            x=corr_across.columns,
            y=corr_across.index,
            colorscale='RdBu',
            zmin=-1, zmax=1,
            colorbar=dict(title="Corr"),
            showscale=True
        ),
        row=2, col=2
    )

    fig.update_layout(
        height=900, width=1200,
        showlegend=False,
        title_text=f"Summary for {series_name}",
        margin=dict(t=60)
    )
    fig.show()

In [38]:
plot_series_summary("Series 2")

# Dashboard

In [ ]:
import dash
from dash import dcc, html, Input, Output, State, dash_table, MATCH, ALL
import dash_daq as daq
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import plotly.graph_objs as go

# --- Defaults ---
def_date_fmt = "%Y-%m-%d"
today = pd.Timestamp.today().normalize()
default_earliest = today - pd.Timedelta(days=90)
default_latest = today

def get_default_timeframes(latest_date):
    return [
        ("Today", latest_date, latest_date),
        ("Last week", latest_date - pd.Timedelta(days=6), latest_date),
        ("Last 4 weeks", latest_date - pd.Timedelta(days=27), latest_date),
        ("Prev 4 weeks", latest_date - pd.Timedelta(days=55), latest_date - pd.Timedelta(days=28)),
        ("Last 8 weeks", latest_date - pd.Timedelta(days=55), latest_date),
        ("Last 12 weeks", latest_date - pd.Timedelta(days=83), latest_date),
    ]

default_tf_colors = {
    "Today": "green",
    "Last week": "blue",
    "Last 4 weeks": "red",
    "Prev 4 weeks": "orange",
    "Last 8 weeks": "yellow",
    "Last 12 weeks": "pink"
}

# --- Helper Functions ---
def generate_timeseries(start_date, end_date, initial_value, params, timeframes, random_state=None):
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    values = np.zeros(len(dates))
    current_value = initial_value
    rng = np.random.default_rng(random_state)
    for tf_label, tf_start, tf_end in timeframes:
        mask = (dates >= tf_start) & (dates <= tf_end)
        n = mask.sum()
        if tf_label in params:
            mu, sigma = params[tf_label]["mean"], params[tf_label]["std"]
        else:
            mu, sigma = 5, 5
        changes = rng.normal(mu, sigma, n)
        tf_values = []
        for change in changes:
            current_value += change
            tf_values.append(current_value)
        values[mask] = tf_values
        if n > 0:
            current_value = tf_values[-1]
    return pd.DataFrame({'Date': dates, 'Value': values})

def Basic_stats(series_df, timeframes):
    stats = []
    for tf_label, tf_start, tf_end in timeframes:
        tf_values = series_df[(series_df['Date'] >= tf_start) & (series_df['Date'] <= tf_end)]['Value']
        if len(tf_values) == 0:
            stats.append([np.nan]*4)
        else:
            stats.append([
                np.quantile(tf_values, 0),
                np.quantile(tf_values, 0.5),
                np.quantile(tf_values, 0.8),
                np.quantile(tf_values, 1)
            ])
    df = pd.DataFrame(stats, columns=['Min', 'Median', 'Quantile 0.8', 'Max'],
                      index=[tf_label for tf_label, _, _ in timeframes])
    return df

def Timeframe_corr(series_df, timeframes):
    """
    Compute pairwise correlation for slices of a time series DataFrame based on equal-span timeframes.

    Parameters:
    - series_df: pd.DataFrame with columns ['Date', 'Value']
    - timeframes: list of tuples -> (label, start_date, end_date)

    Returns:
    - pd.DataFrame containing the correlation matrix of valid timeframe slices
    """

    # Ensure 'Date' column is datetime
    series_df = series_df.copy()
    series_df['Date'] = pd.to_datetime(series_df['Date'])

    slices = {}

    valid_timeframes = None
    duration_list = []

    for label, start, end in timeframes:
        start = pd.to_datetime(start)
        end = pd.to_datetime(end)
        mask = (series_df['Date'] >= start) & (series_df['Date'] <= end)
        values = series_df.loc[mask, 'Value'].values

        duration = (end - start).days + 1  # inclusive
        duration_list.append(duration)
        # skip non-uniform spans or missing days
        slices[label] = values

    same_durations = [duration_list.count(d) > 1 for d in duration_list]
    copy_of_slice_keys = list(slices.keys()).copy()
    for label, duration_boolean in zip(copy_of_slice_keys, same_durations):
        if not duration_boolean:
            del slices[label]
    if sum(same_durations) < 2:
        corr_matrix = pd.DataFrame(data=['No timeframes with same duration span (>1 day)'])
        return corr_matrix
    else:
        # Create a DataFrame from slices
        slice_df = pd.DataFrame(slices)
        corr_matrix = slice_df.corr()
        corr_matrix.index = slices.keys()
        corr_matrix.columns = slices.keys()

    return corr_matrix

def Series_corr(series_dict, timeframes, tf_label):
    tf_idx = [i for i, (lbl, _, _) in enumerate(timeframes) if lbl == tf_label]
    if not tf_idx:
        return pd.DataFrame()
    tf_idx = tf_idx[0]
    vals = []
    names = []
    min_len = None
    for name, df in series_dict.items():
        tf_start, tf_end = timeframes[tf_idx][1], timeframes[tf_idx][2]
        # Ensure Date column is datetime
        if not np.issubdtype(df['Date'].dtype, np.datetime64):
            df = df.copy()
            df['Date'] = pd.to_datetime(df['Date'])
        v = df[(df['Date'] >= tf_start) & (df['Date'] <= tf_end)]['Value'].values
        if len(v) == 0:
            continue
        if min_len is None or len(v) < min_len:
            min_len = len(v)
        vals.append(v)
        names.append(name)
    if len(vals) < 2 or min_len is None or min_len == 0:
        return pd.DataFrame(np.nan, index=names, columns=names)
    trimmed = [arr[:min_len] for arr in vals]
    data = np.column_stack(trimmed)
    valid_cols = [i for i in range(data.shape[1]) if np.std(data[:, i]) > 1e-8]
    if len(valid_cols) < 2:
        return pd.DataFrame(np.nan, index=[names[i] for i in valid_cols], columns=[names[i] for i in valid_cols])
    data = data[:, valid_cols]
    valid_names = [names[i] for i in valid_cols]
    corr = np.corrcoef(data, rowvar=False)
    return pd.DataFrame(corr, index=valid_names, columns=valid_names)

# --- UI Helper Functions ---
def make_timeframe_picker(tf_label, tf_start, tf_end, color, idx, editable=True):
    return html.Div([
        html.Label(tf_label, style={'fontWeight': 'bold', 'marginRight': '10px'}),
        dcc.Input(id={'type': 'tf-label', 'index': idx}, value=tf_label, style={'width': '100px', 'marginRight': '10px'}, disabled=not editable),
        dcc.DatePickerRange(
            id={'type': 'tf-date', 'index': idx},
            start_date=tf_start.strftime(def_date_fmt),
            end_date=tf_end.strftime(def_date_fmt),
            display_format='YYYY-MM-DD',
            disabled=not editable
        ),
        daq.ColorPicker(
            id={'type': 'tf-color', 'index': idx},
            value={'hex': color},
            size=164,
            disabled=not editable,
            style={'display': 'inline-block', 'marginLeft': '10px'}
        ),
        html.Button('Delete', id={'type': 'tf-delete', 'index': idx}, n_clicks=0, style={'marginLeft': '10px', 'display': 'inline-block'} if editable else {'display': 'none'})
    ], style={'marginBottom': '10px', 'display': 'flex', 'alignItems': 'center'})

def make_series_param_editor(series_name, params, idx, editable=True):
    tf_label = list(params['params'].keys())[0]
    mean = params['params'][tf_label]['mean']
    std = params['params'][tf_label]['std']
    return html.Div([
        html.Label(f"{series_name} Initial Value:"),
        dcc.Input(id={'type': 'series-init', 'index': idx}, type='number', value=params['initial_value'], style={'width': '80px', 'marginRight': '10px'}, disabled=not editable),
        html.Label("Daily Change Mean:"),
        dcc.Input(id={'type': 'series-mean', 'index': idx}, type='number', value=mean, style={'width': '60px', 'marginRight': '10px'}, disabled=not editable),
        html.Label("Std:"),
        dcc.Input(id={'type': 'series-std', 'index': idx}, type='number', value=std, style={'width': '60px', 'marginRight': '10px'}, disabled=not editable),
        html.Button('Delete', id={'type': 'series-delete', 'index': idx}, n_clicks=0, style={'marginLeft': '10px'} if editable else {'display': 'none'})
    ], style={'marginBottom': '10px', 'display': 'flex', 'alignItems': 'center'})

# --- App Layout ---
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Store(id='series-store', data={
        "Series 1": {
            "initial_value": 500,
            "params": {
                "Last 12 weeks": {"mean": 0, "std": 5}
            }
        },
        "Series 2": {
            "initial_value": 900,
            "params": {
                "Last 12 weeks": {"mean": 0, "std": 10}
            }
        }
    }),
    dcc.Store(id='tf-store', data=[
        {
            "label": lbl,
            "start": tf_start.strftime(def_date_fmt),
            "end": tf_end.strftime(def_date_fmt),
            "color": default_tf_colors.get(lbl, "#b3e2cd")
        }
        for lbl, tf_start, tf_end in get_default_timeframes(default_latest)
    ]),
    dcc.Store(id='editing-mode', data=True),
    html.H2("Timeseries Dashboard"),
    html.Div([
        html.Label("Earliest Date:"),
        dcc.DatePickerSingle(id='earliest-date', date=default_earliest.strftime(def_date_fmt)),
        html.Label("Latest Date:", style={'marginLeft': '20px'}),
        dcc.DatePickerSingle(id='latest-date', date=default_latest.strftime(def_date_fmt)),
    ], style={'marginBottom': '20px'}),
    html.H4("Custom Timeframes"),
    html.Div(id='custom-timeframes'),
    html.Button('Add Custom Timeframe', id='add-tf', n_clicks=0, style={'marginBottom': '20px'}),
    html.H4("Series Parameters"),
    html.Div(id='series-params'),
    html.Button('Add Series', id='add-series', n_clicks=0, style={'marginBottom': '20px'}),
    html.Button('Done', id='done-btn', n_clicks=0, style={'marginBottom': '20px'}),
    html.Div([
        html.Label("Select Series:"),
        dcc.Dropdown(id='series-dropdown',style={'width': '200px'}),
        html.Label("Select Timeframe for Series Corr:", style={'marginLeft': '20px'}),
        dcc.Dropdown(id='tf-dropdown',style={'width': '200px'}),
    ], style={'marginBottom': '20px', 'display': 'flex', 'alignItems': 'center'}),
    dcc.Loading([
        dcc.Graph(id='plot-1', style={'display': 'inline-block', 'width': '48%'}),
        dcc.Graph(id='plot-2', style={'display': 'inline-block', 'width': '48%'}),
        dcc.Graph(id='plot-3', style={'display': 'inline-block', 'width': '48%'}),
        dcc.Graph(id='plot-4', style={'display': 'inline-block', 'width': '48%'}),
    ])
])

# --- Editing Mode Callback ---
@app.callback(
    Output('editing-mode', 'data'),
    Input('add-tf', 'n_clicks'),
    Input('add-series', 'n_clicks'),
    Input({'type': 'tf-delete', 'index': ALL}, 'n_clicks'),
    Input({'type': 'series-delete', 'index': ALL}, 'n_clicks'),
    Input('done-btn', 'n_clicks'),
    State('editing-mode', 'data'),
    prevent_initial_call=True
)
def set_editing_mode(add_tf, add_series, del_tf, del_series, done, editing):
    ctx = dash.callback_context
    if not ctx.triggered:
        return editing
    btn = ctx.triggered[0]['prop_id'].split('.')[0]
    if btn == 'done-btn':
        return False
    return True

# --- Timeframes Editor Callback ---
@app.callback(
    Output('custom-timeframes', 'children'),
    Output('tf-store', 'data'),
    Input('add-tf', 'n_clicks'),
    Input({'type': 'tf-delete', 'index': ALL}, 'n_clicks'),
    Input({'type': 'tf-label', 'index': ALL}, 'value'),
    Input({'type': 'tf-date', 'index': ALL}, 'start_date'),
    Input({'type': 'tf-date', 'index': ALL}, 'end_date'),
    Input({'type': 'tf-color', 'index': ALL}, 'value'),
    State('tf-store', 'data'),
    State('editing-mode', 'data'),
    prevent_initial_call=True
)
def update_timeframes(add_tf_clicks, del_clicks, labels, starts, ends, colors, tf_data, editing_mode):
    ctx = dash.callback_context
    triggered = ctx.triggered[0]['prop_id'].split('.')[0]
    tf_data = tf_data or []
    # Handle delete
    if 'tf-delete' in triggered:
        idx = int(eval(triggered)['index'])
        tf_data = [tf for i, tf in enumerate(tf_data) if i != idx]
    # Handle add
    elif 'add-tf' in triggered:
        tf_data = tf_data + [{
            "label": f"Custom TF {len(tf_data)+1}",
            "start": default_earliest.strftime(def_date_fmt),
            "end": default_latest.strftime(def_date_fmt),
            "color": "#b3e2cd"
        }]
    # Handle edits
    else:
        for i in range(len(tf_data)):
            tf_data[i]['label'] = labels[i]
            tf_data[i]['start'] = pd.to_datetime(starts[i])
            tf_data[i]['end'] = pd.to_datetime(ends[i])
            tf_data[i]['color'] = colors[i]['hex'] if isinstance(colors[i], dict) else colors[i]
    # Render
    if editing_mode:
        children = [
            make_timeframe_picker(tf['label'], pd.to_datetime(tf['start']), pd.to_datetime(tf['end']), tf['color'], idx, editable=True)
            for idx, tf in enumerate(tf_data)
        ]
    else:
        children = []
    return children, tf_data

# --- Series Editor Callback ---
@app.callback(
    Output('series-params', 'children'),
    Output('series-store', 'data'),
    Input('add-series', 'n_clicks'),
    Input({'type': 'series-delete', 'index': ALL}, 'n_clicks'),
    Input({'type': 'series-init', 'index': ALL}, 'value'),
    Input({'type': 'series-mean', 'index': ALL}, 'value'),
    Input({'type': 'series-std', 'index': ALL}, 'value'),
    State('series-store', 'data'),
    State('tf-store', 'data'),
    State('editing-mode', 'data'),
    prevent_initial_call=True
)
def update_series(add_series_clicks, del_clicks, inits, means, stds, series_data, tf_data, editing_mode):
    ctx = dash.callback_context
    triggered = ctx.triggered[0]['prop_id'].split('.')[0]
    series_data = series_data or {}
    tf_data = tf_data or []
    tf_label = tf_data[-1]['label'] if tf_data else "Last 12 weeks"
    # Handle delete
    if 'series-delete' in triggered:
        idx = int(eval(triggered)['index'])
        keys = list(series_data.keys())
        if idx < len(keys):
            del series_data[keys[idx]]
    # Handle add
    elif 'add-series' in triggered:
        new_idx = len(series_data) + 1
        series_data[f"Series {new_idx}"] = {
            "initial_value": 20,
            "params": {
                tf_label: {"mean": 4, "std": 10}
            }
        }
    # Handle edits
    else:
        keys = list(series_data.keys())
        for i, k in enumerate(keys):
            if i < len(inits):
                series_data[k]['initial_value'] = inits[i]
                series_data[k]['params'][tf_label] = {"mean": means[i], "std": stds[i]}
    # Render
    if editing_mode:
        children = [
            make_series_param_editor(k, v, idx, editable=True)
            for idx, (k, v) in enumerate(series_data.items())
        ]
    else:
        children = []
    return children, series_data

# --- Dashboard Update Callback ---
@app.callback(
    Output('series-dropdown', 'options'),
    Output('series-dropdown', 'value'),
    Output('tf-dropdown', 'options'),
    Output('tf-dropdown', 'value'),
    Output('plot-1', 'figure'),
    Output('plot-2', 'figure'),
    Output('plot-3', 'figure'),
    Output('plot-4', 'figure'),
    Input('earliest-date', 'date'),
    Input('latest-date', 'date'),
    Input('series-dropdown', 'value'),
    Input('tf-dropdown', 'value'),
    Input('series-store', 'data'),
    Input('tf-store', 'data'),
)
def update_dashboard(earliest_date, latest_date, selected_series, selected_tf, series_data, tf_data):
    earliest = pd.to_datetime(earliest_date)
    latest = pd.to_datetime(latest_date)
    timeframes = [
        (tf['label'], pd.to_datetime(tf['start']), pd.to_datetime(tf['end']))
        for tf in tf_data
    ]
    tf_labels = [lbl for lbl, _, _ in timeframes]
    tf_colors = [tf['color'] for tf in tf_data]

    series_dict = {}
    for i, (name, params) in enumerate(series_data.items()):
        series_seed = abs(hash(name)) % (2**32)
        df = generate_timeseries(
            earliest, latest,
            params['initial_value'],
            params['params'],
            timeframes,
            random_state=series_seed
        )
        series_dict[name] = df

    series_options = [{'label': k, 'value': k} for k in series_dict.keys()]
    if not selected_series or selected_series not in series_dict:
        selected_series = series_options[0]['value'] if series_options else None
    tf_options = [{'label': lbl, 'value': lbl} for lbl in tf_labels]
    if not selected_tf or selected_tf not in tf_labels:
        selected_tf = tf_labels[-1] if tf_labels else None

    if not selected_series or not timeframes:
        return series_options, selected_series, tf_options, selected_tf, {}, {}, {}, {}
    df_selected = series_dict[selected_series]
    stats_df = Basic_stats(df_selected, timeframes)
    corr_df = Timeframe_corr(df_selected, timeframes)
    series_corr_df = Series_corr(series_dict, timeframes, selected_tf)

    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(
        x=df_selected['Date'], y=df_selected['Value'],
        mode='lines', name=selected_series
    ))
    for i, (tf_label, tf_start, tf_end) in enumerate(timeframes):
        fig1.add_vrect(
            x0=tf_start, x1=tf_end,
            fillcolor=tf_colors[i], opacity=0.2, line_width=0,
            annotation_text=tf_label, annotation_position="top left"
        )
    fig1.update_layout(
        title=f"Timeseries: {selected_series}",
        xaxis_title="Date", yaxis_title="Value",
        showlegend=False, height=400
    )

    stats_df_fmt = stats_df.round(2)
    cell_colors = [tf_colors] + [['lavender'] * len(stats_df)] * (len(stats_df.columns))
    fig2 = go.Figure(data=[go.Table(
        header=dict(
            values=["Timeframe"] + list(stats_df_fmt.columns),
            fill_color='paleturquoise',
            align='left'
        ),
        cells=dict(
            values=[[idx for idx in stats_df_fmt.index]] + [stats_df_fmt[col].tolist() for col in stats_df_fmt.columns],
            fill_color=cell_colors,
            format=[None] + [".2f"] * len(stats_df_fmt.columns),
            align='left'
        )
    )])
    fig2.update_layout(
        title=f"Basic Stats: {selected_series}",
        height=400
    )

    corr_df_fmt = corr_df.round(2)
    fig3 = go.Figure(data=go.Heatmap(
        z=corr_df_fmt.values,
        x=corr_df_fmt.columns,
        y=corr_df_fmt.index,
        colorscale='RdBu', zmin=-1, zmax=1,
        colorbar=dict(title="Corr"),
        hovertemplate='Corr: %{z:.2f}<extra></extra>'
    ))
    fig3.update_layout(
        title=f"Timeframe Corr: {selected_series}",
        height=400
    )

    series_corr_fmt = series_corr_df.round(2)
    fig4 = go.Figure(data=go.Heatmap(
        z=series_corr_fmt.values,
        x=series_corr_fmt.columns,
        y=series_corr_fmt.index,
        colorscale='RdBu', zmin=-1, zmax=1,
        colorbar=dict(title="Corr"),
        hovertemplate='Corr: %{z:.2f}<extra></extra>'
    ))
    fig4.update_layout(
        title=f"Series Corr: {selected_tf}",
        height=400
    )

    return series_options, selected_series, tf_options, selected_tf, fig1, fig2, fig3, fig4

# To run the app:
if __name__ == "__main__":
    app.run(host='localhost', port=8501, debug=True)